In [1]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import pandas as pd
import csv


# Import matplotlib libraries
# from matplotlib import pyplot as plt
# from matplotlib.collections import LineCollection
# import matplotlib.patches as patches

# import imageio
# from IPython.display import HTML, display


In [2]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [3]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[6:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/train/*")
    save_dir = "dataset/train"
    

    for path in video_paths:
        save_frame(path, save_dir, gap=30)
    
    video_paths = glob("Yoga_Vid_Collected/test/*")
    save_dir = "dataset/test"
    
    for path in video_paths:
        save_frame(path, save_dir, gap=30)

train\abhay_bhuj
abhay_bhuj
dataset/train\Bhujangasana\abhay_bhuj
train\abhay_padam
abhay_padam
dataset/train\Padamasana\abhay_padam
train\abhay_shav
abhay_shav
dataset/train\Shavasana\abhay_shav
train\abhay_tada
abhay_tada
dataset/train\Tadasana\abhay_tada
train\abhay_trik
abhay_trik
dataset/train\Trikonasana\abhay_trik
train\abhay_vriksh
abhay_vriksh
dataset/train\Vrikshasana\abhay_vriksh
train\ameya_bhuj
ameya_bhuj
dataset/train\Bhujangasana\ameya_bhuj
train\ameya_padam
ameya_padam
dataset/train\Padamasana\ameya_padam
train\ameya_shav
ameya_shav
dataset/train\Shavasana\ameya_shav
train\ameya_tadasan
ameya_tadasan
dataset/train\Tadasana\ameya_tadasan
train\ameya_trikonasana
ameya_trikonasana
dataset/train\Trikonasana\ameya_trikonasana
train\ameya_vriksh
ameya_vriksh
dataset/train\Vrikshasana\ameya_vriksh
train\bhumi_bhuj
bhumi_bhuj
dataset/train\Bhujangasana\bhumi_bhuj
train\bhumi_padam
bhumi_padam
dataset/train\Padamasana\bhumi_padam
train\bhumi_shav
bhumi_shav
dataset/train\Shavasa

In [ ]:
def keypts_abs(keypoints):
  kpts_x = keypoints[0, 0, :, 0]
  kpts_y = keypoints[0, 0, :, 1]
  kpts_scores = keypoints[0, 0, :, 2]

  print(kpts_x.shape)
  kpts_absolute_xy = np.stack(
      [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)

  kpts_absolute_xy.shape

  keypoint_threshold=0.2
  kpts_above_thresh_absolute = kpts_absolute_xy[
      kpts_scores > keypoint_threshold, :]
  return kpts_above_thresh_absolute

In [13]:
data=[]
KEYPOINT_DICT = [
    'nose',
    'left_eye',
    'right_eye',
    'left_ear',
    'right_ear',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle'
]
for p in KEYPOINT_DICT:
        data.append(p + "_x")
        data.append(p + "_y")
#         data.append(p + "_vis")
data.append( "label")
with open('testfile.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(data)
# Load the input image.
  
def image_to_movenet(path,keypts_labels):
    
    if (path.find("Bhujangasana") != -1) :
          label="Bhujangasana"  
    elif (path.find("Padamasana") != -1):
          label="Padamasana"
    elif (path.find("Shavasana") != -1):
          label="Shavasana"
    elif (path.find("Trikonasana") != -1):
          label="Trikonasana"
    elif (path.find("Vrikshasana") != -1):
          label="Vrikshasana"
    elif (path.find("Tadasana") != -1):
          label="Tadasana"
        
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    kpts_x = keypoints[0, 0, :, 0]
    kpts_y = keypoints[0, 0, :, 1]
    kpts_scores = keypoints[0, 0, :, 2]
    
    keypoints_tup=(keypoints,label)
    keypts_labels.append(keypoints_tup)
    print(keypoints)
    
    

     # Empty dataset
    t_x=kpts_x.numpy()
    t_y=kpts_y.numpy()
    
    tn=[]
#     tn=['nose_x', 'nose_y', 'nose_vis', 'left_eye_x', 'left_eye_y', 'left_eye_vis', 'right_eye_x', 'right_eye_y', 'right_eye_vis', 'left_ear_x', 'left_ear_y', 'left_ear_vis', 'right_ear_x', 'right_ear_y', 'right_ear_vis', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_vis', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_vis', 'left_elbow_x', 'left_elbow_y', 'left_elbow_vis', 'right_elbow_x', 'right_elbow_y', 'right_elbow_vis', 'left_wrist_x', 'left_wrist_y', 'left_wrist_vis', 'right_wrist_x', 'right_wrist_y', 'right_wrist_vis', 'left_hip_x', 'left_hip_y', 'left_hip_vis', 'right_hip_x', 'right_hip_y', 'right_hip_vis', 'left_knee_x', 'left_knee_y', 'left_knee_vis', 'right_knee_x', 'right_knee_y', 'right_knee_vis', 'left_ankle_x', 'left_ankle_y', 'left_ankle_vis', 'right_ankle_x', 'right_ankle_y', 'right_ankle_vis', 'label']
   
    for i in range(0,17):
        tn.append(t_x[i])
        tn.append(t_y[i])
    tn.append(label)
#      #convert to Numpy array

    
    with open('testfile.csv','a') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(tn)


In [14]:
import os
import time

path_lst=[]
keypts_labels_train=[]
keypts_labels_test=[]


class_list=os.listdir('dataset/train')
for name in class_list:
    for image in os.listdir(f"dataset/train/{name}"):
        for item in os.listdir(f"dataset/train/{name}/{image}"):
             path_lst.append(os.path.abspath(f"dataset/train/{name}/{image}/{item}"))



path_lst = sorted(path_lst,key=os.path.getmtime)
for path in path_lst:
    image_to_movenet(path,keypts_labels_train)

# class_list=os.listdir('dataset/test')
# for name in class_list:
#     for image in os.listdir(f"dataset/test/{name}"):
#         for item in os.listdir(f"dataset/test/{name}/{image}"):
#              path_lst.append(os.path.abspath(f"dataset/test/{name}/{image}/{item}"))



# path_lst = sorted(path_lst,key=os.path.getmtime)
# for path in path_lst:
#     image_to_movenet(path,keypts_labels_test)    


tf.Tensor(
[[[[0.78621894 0.87021947 0.5061497 ]
   [0.7801042  0.87738633 0.45931965]
   [0.77937704 0.8753842  0.5093354 ]
   [0.7547425  0.86347234 0.5314945 ]
   [0.75545913 0.86047244 0.50789946]
   [0.7497095  0.8122004  0.50559   ]
   [0.7793317  0.81107634 0.6936564 ]
   [0.7556961  0.68825954 0.25572452]
   [0.7984366  0.69402623 0.5690342 ]
   [0.78506917 0.6002404  0.28858364]
   [0.8078833  0.5847824  0.49464497]
   [0.7335595  0.5786643  0.6360946 ]
   [0.74406546 0.5759433  0.59036946]
   [0.7438633  0.43917832 0.64149344]
   [0.77175796 0.43098912 0.42472115]
   [0.72000456 0.33020774 0.31878906]
   [0.7472891  0.30311382 0.38912243]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.7752273  0.87912154 0.5893799 ]
   [0.7615156  0.88218987 0.62062436]
   [0.7638898  0.880481   0.5920632 ]
   [0.74586654 0.85848904 0.55560553]
   [0.7511187  0.8573169  0.65656614]
   [0.74968785 0.8038533  0.5704179 ]
   [0.78395736 0.803237   0.5592039 ]
   [0.7541457  0.68630564 

tf.Tensor(
[[[[0.5244836  0.7302153  0.6507487 ]
   [0.5105212  0.72556615 0.59186506]
   [0.51455796 0.71536946 0.58302224]
   [0.5203596  0.6904832  0.21027195]
   [0.529009   0.6772463  0.66361296]
   [0.5842067  0.661056   0.5925237 ]
   [0.6067718  0.685757   0.39616084]
   [0.71252596 0.6790048  0.35192838]
   [0.72365165 0.69114655 0.47916314]
   [0.78010154 0.6787312  0.20801213]
   [0.81422764 0.6948887  0.30799007]
   [0.7263406  0.568563   0.40970242]
   [0.7387674  0.5743927  0.5166811 ]
   [0.739755   0.43042475 0.41286838]
   [0.76846474 0.42942882 0.49272403]
   [0.7361138  0.3117247  0.2631322 ]
   [0.75826854 0.28226337 0.49637276]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.51316535 0.7342632  0.38710842]
   [0.5015684  0.7240874  0.28439587]
   [0.5051923  0.7202534  0.4597895 ]
   [0.5132449  0.6903031  0.13038912]
   [0.52276444 0.686136   0.5452082 ]
   [0.5850929  0.66235816 0.632719  ]
   [0.6040131  0.68628997 0.6280353 ]
   [0.71169573 0.67806584 

tf.Tensor(
[[[[0.5286296  0.681224   0.67578125]
   [0.51796705 0.69112676 0.5294875 ]
   [0.5178034  0.6676752  0.50404346]
   [0.5212086  0.7071815  0.64408094]
   [0.5253265  0.6487341  0.73896015]
   [0.58807087 0.7295479  0.7701299 ]
   [0.5894759  0.6163665  0.7077286 ]
   [0.64945626 0.76341623 0.43996957]
   [0.6696886  0.59938025 0.54621065]
   [0.7238321  0.8015199  0.46202496]
   [0.73548377 0.5860246  0.37364072]
   [0.72344095 0.71380097 0.58335996]
   [0.7316827  0.6453544  0.54801357]
   [0.75192344 0.77946204 0.3118272 ]
   [0.7484895  0.59763205 0.3279435 ]
   [0.79844266 0.72785276 0.186863  ]
   [0.8071951  0.68692946 0.16663468]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.52838284 0.6829867  0.652845  ]
   [0.51805305 0.6930392  0.5795988 ]
   [0.51745796 0.6701096  0.30102462]
   [0.52174026 0.70898575 0.6915128 ]
   [0.5250298  0.6498686  0.7559503 ]
   [0.5875078  0.73072195 0.72451687]
   [0.5886425  0.61736906 0.71626556]
   [0.64952815 0.7641465  

tf.Tensor(
[[[[0.5540178  0.6850659  0.6442462 ]
   [0.5415236  0.6967227  0.72967607]
   [0.54193676 0.67463636 0.56040317]
   [0.54557556 0.7122915  0.46896967]
   [0.5481274  0.6553926  0.23737144]
   [0.6144418  0.735418   0.56872684]
   [0.6097601  0.6205303  0.418178  ]
   [0.69790787 0.7182584  0.4461542 ]
   [0.70610434 0.6065579  0.56863153]
   [0.76251763 0.6884734  0.55277175]
   [0.7707567  0.60895807 0.29459715]
   [0.7406585  0.69269747 0.46301955]
   [0.7436323  0.6361949  0.5375603 ]
   [0.7572376  0.79419035 0.6011225 ]
   [0.7646064  0.58704185 0.35806233]
   [0.7913161  0.6716158  0.31592873]
   [0.80358344 0.6434697  0.30383265]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.55681646 0.675111   0.6052823 ]
   [0.5427679  0.68757445 0.5269259 ]
   [0.54358613 0.66514456 0.6242773 ]
   [0.5470591  0.7073636  0.42297292]
   [0.5488     0.6483659  0.45160678]
   [0.61177945 0.7277104  0.6696865 ]
   [0.6012988  0.61370856 0.77738476]
   [0.6977126  0.71080095 

tf.Tensor(
[[[[0.58929753 0.60549337 0.49938428]
   [0.5805631  0.62363434 0.48670265]
   [0.58105123 0.6169076  0.4594807 ]
   [0.6025373  0.6559831  0.47580197]
   [0.6031635  0.64027375 0.15685162]
   [0.6713386  0.65971696 0.4814104 ]
   [0.66433066 0.62969023 0.5940145 ]
   [0.7884567  0.6561297  0.5501876 ]
   [0.7609634  0.60899955 0.1364168 ]
   [0.8142711  0.5518296  0.18573368]
   [0.7907987  0.53357863 0.20707014]
   [0.7774448  0.49358898 0.38009858]
   [0.75692207 0.48006406 0.48087892]
   [0.68448365 0.3848422  0.46606928]
   [0.6810121  0.38408142 0.48138246]
   [0.76268846 0.24593411 0.25651473]
   [0.7586593  0.23627032 0.25503582]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.70331955 0.8116763  0.44222847]
   [0.71534896 0.8290531  0.4120288 ]
   [0.7084458  0.8299857  0.51089156]
   [0.7486818  0.8243875  0.44114006]
   [0.7457883  0.82890654 0.6638011 ]
   [0.7724884  0.7786394  0.74508286]
   [0.73358065 0.7550327  0.5683042 ]
   [0.8060091  0.66398084 

...

tf.Tensor(
[[[[0.24858418 0.53072155 0.6388502 ]
   [0.23645718 0.54368687 0.6120617 ]
   [0.2349799  0.516301   0.5327545 ]
   [0.24823418 0.5624907  0.672364  ]
   [0.2473558  0.49764442 0.68596315]
   [0.31826702 0.59652793 0.83291477]
   [0.3191749  0.46452013 0.9064255 ]
   [0.4291819  0.6389334  0.7241437 ]
   [0.42319486 0.42967    0.7554046 ]
   [0.3780733  0.5568236  0.6987887 ]
   [0.37943518 0.51116747 0.6473181 ]
   [0.5050824  0.57248104 0.8491025 ]
   [0.51802546 0.4947503  0.89505446]
   [0.59868705 0.6980171  0.9057348 ]
   [0.7028313  0.5054885  0.65798825]
   [0.6469441  0.55752057 0.73527145]
   [0.84108365 0.53501743 0.78668976]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.24930532 0.5160697  0.5478143 ]
   [0.23604192 0.53004086 0.38611975]
   [0.23718035 0.50366944 0.75631917]
   [0.24714443 0.5561306  0.81258196]
   [0.25143987 0.49396932 0.6807054 ]
   [0.31707406 0.5940585  0.84261423]
   [0.3212247  0.45961604 0.9200847 ]
   [0.42653397 0.6362317  

tf.Tensor(
[[[[0.2515627  0.55043036 0.59644026]
   [0.2420108  0.5656589  0.7411275 ]
   [0.23896806 0.53838146 0.79011124]
   [0.25664198 0.58627146 0.8893863 ]
   [0.2525267  0.52548987 0.7630032 ]
   [0.32630002 0.61313367 0.7316807 ]
   [0.31388682 0.47955373 0.8811147 ]
   [0.43504673 0.64317214 0.71000814]
   [0.4173544  0.43673024 0.54824924]
   [0.38171995 0.56189036 0.7008109 ]
   [0.38280106 0.5225841  0.71306485]
   [0.5060265  0.57745993 0.8876762 ]
   [0.51475966 0.49990052 0.83566785]
   [0.6010363  0.69727707 0.936713  ]
   [0.7038646  0.51013684 0.63808966]
   [0.6487471  0.56266993 0.7909076 ]
   [0.84784186 0.5357682  0.75484705]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.25136867 0.5578787  0.5669112 ]
   [0.2423243  0.57432973 0.8026196 ]
   [0.23830844 0.5488441  0.61543524]
   [0.2591974  0.5913812  0.7483978 ]
   [0.25403467 0.53209525 0.6439218 ]
   [0.32648566 0.6181107  0.7719828 ]
   [0.3142602  0.48799008 0.87803686]
   [0.43603826 0.64590824 

tf.Tensor(
[[[[0.49317223 0.6753734  0.67220926]
   [0.4793946  0.66825247 0.6046004 ]
   [0.48033482 0.66462034 0.7808726 ]
   [0.4848113  0.63724303 0.39936835]
   [0.48946285 0.6286491  0.6726989 ]
   [0.5350757  0.60987025 0.63952905]
   [0.53978133 0.6070256  0.5100537 ]
   [0.6592127  0.6010889  0.60277605]
   [0.66687346 0.5966064  0.60659325]
   [0.74862087 0.6012993  0.36039317]
   [0.76457334 0.5977098  0.75626993]
   [0.65584975 0.4644732  0.63281626]
   [0.66099787 0.45536274 0.6827701 ]
   [0.6832034  0.32901898 0.6901423 ]
   [0.69670683 0.32722235 0.47423223]
   [0.67540824 0.19927946 0.6026442 ]
   [0.68624383 0.18243533 0.7305666 ]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.50658727 0.61571306 0.68018454]
   [0.49565873 0.6326571  0.7873446 ]
   [0.49275482 0.60664624 0.7175592 ]
   [0.4996499  0.65096194 0.6185989 ]
   [0.49313134 0.5889248  0.5447291 ]
   [0.5418409  0.6421134  0.6242534 ]
   [0.549748   0.57056767 0.54339755]
   [0.6356449  0.6233676  

tf.Tensor(
[[[[0.509911   0.65551436 0.40809813]
   [0.49028605 0.6420939  0.29184198]
   [0.49824986 0.6581696  0.42151776]
   [0.48645955 0.60921973 0.52958256]
   [0.50219846 0.6517019  0.53350276]
   [0.5273158  0.58355856 0.666465  ]
   [0.54723424 0.6249918  0.5984722 ]
   [0.65028656 0.5984621  0.40508738]
   [0.66451937 0.61184597 0.82086957]
   [0.70770687 0.5976711  0.2082816 ]
   [0.7580763  0.59950525 0.5663122 ]
   [0.6576276  0.46539554 0.704697  ]
   [0.6669154  0.47432885 0.6479894 ]
   [0.68651444 0.33101052 0.62397105]
   [0.70277065 0.33563074 0.6403885 ]
   [0.67522305 0.18680252 0.66168356]
   [0.6878963  0.17796521 0.58473295]]]], shape=(1, 1, 17, 3), dtype=float32)
tf.Tensor(
[[[[0.51703167 0.65647113 0.44055408]
   [0.4970548  0.6461476  0.4868087 ]
   [0.5049842  0.6622156  0.6380385 ]
   [0.48735818 0.6196435  0.6245886 ]
   [0.50815666 0.65628684 0.54448783]
   [0.529246   0.5903954  0.5798057 ]
   [0.5486053  0.6163777  0.66523826]
   [0.6477023  0.591464   

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for C:\Users\radha\AppData\Local\Temp\tfhub_modules\f729a5f3231391676ca61cc7ab789993549d8bca\variables\variables
 If trying to load on a different device from the computational device, consider using setting the `experimental_io_device` option on tf.saved_model.LoadOptions to the io_device such as '/job:localhost'.